In [350]:
import rasterio
import geopandas as gpd
import numpy as np
from skimage.graph.mcp import MCP_Geometric
from onstove.raster import *
from shapely.geometry import shape

In [351]:
# This map represents the travel speed from this allocation process, expressed in units of minutes 
# required to travel one metre (Raster).
friction = r"C:\Proposals\CCA\Bimass_CODE_TEST_LAYERS\friction.tif"
# Startpoints (Raster)
forest_cover = r"C:\Proposals\CCA\Bimass_CODE_TEST_LAYERS\Forest.tif"
#AOI (Polygon)
mask = r"C:\Proposals\CCA\gadm36_NPL_shp\gadm36_NPL_0.shp"
# Potential restriction areas (Polygon)
restrictions = r"C:\Proposals\CCA\Bimass_CODE_TEST_LAYERS\protected.shp"
restricted = r"C:\Proposals\CCA\Bimass_CODE_TEST_LAYERS\restricted_area.tif"
#new friction based on start points (Raster)
travel_time = r"C:\Proposals\CCA\Bimass_CODE_TEST_LAYERS\travel_time.tif"

In [352]:
def align_raster(raster_1, raster_2, method='nearest', compression='NONE'):
    with rasterio.open(raster_1) as src:
        raster_1_meta = src.meta
    with rasterio.open(raster_2) as src:
        raster_2 = src.read(1)
        raster_2_meta = src.meta

    out_meta = raster_1_meta.copy()
    out_meta.update({
        'transform': raster_1_meta['transform'],
        'crs': raster_1_meta['crs'],
        'compress': compression
    })    
    destination = np.full((raster_1_meta['height'], raster_1_meta['width']), raster_2_meta['nodata'])
    reproject(
            source=raster_2,
            destination=destination,
            src_transform=raster_2_meta['transform'],
            src_crs=raster_2_meta['crs'],
            dst_transform=raster_1_meta['transform'],
            dst_crs=raster_1_meta['crs'],
            resampling=Resampling[method])
    return destination, out_meta

In [353]:
def polygonize(raster, mask = None):
    with rasterio.Env():
        if type(raster) == str:
            with rasterio.open(raster) as raster:
                raster = raster.read(1)
                raster = raster.astype('float32')
            
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
        shapes(raster, mask=mask, transform=src.transform)))
    
    geoms = list(results)
    polygon  = gpd.GeoDataFrame.from_features(geoms)
    return polygon

In [354]:
#Generated a restriction mask
mask_raster(forest_cover, restrictions, restricted, nodata=0, compression='NONE')
new_restrictions, new_restrictions_meta = align_raster(friction, restricted, method='nearest', compression='NONE')

In [355]:
#Allign the forest raster with the friction raster
new_forest, new_forest_meta = align_raster(friction, forest_cover, method='nearest', compression='NONE')
new_forest = new_forest+new_restrictions
new_forest[new_forest == 1] = 0
new_forest[new_forest != 0] = 1
rows, cols = np.where(new_forest == 0)

In [356]:
# Transform friction raster to km and hour instead of minutes and meter
with rasterio.open(friction) as fric:
    fric_meta = fric.meta
    fric_val = fric.read(1)
    fric_val = fric_val*1000/60

In [357]:
# Create a new friction map by multiplying the firction map with the new forest cover map
new_fric = fric_val*new_forest

In [358]:
# Minimum cost path betweenn all nodes
mcp = MCP_Geometric(new_fric, fully_connected=True)

In [359]:
#Adding statvalues
pointlist = []
i = 0
for row in rows:
    positiions = [rows[i],cols[i]]
    pointlist.append(positiions)
    i = i+1

In [360]:
#minimum cost
cumulative_costs, traceback = mcp.find_costs(starts=pointlist)
cumulative_costs[np.where(cumulative_costs==float('inf'))] = np.nan

In [361]:
new_dataset = rasterio.open(travel_time, 'w', driver='GTiff',
                            height = src.shape[0], width = src.shape[1],
                            count=1, dtype="float64",
                            crs=src.crs, transform = src.transform)

new_dataset.write(cumulative_costs, 1)
new_dataset.close()